Trước khi run pip, cài đặt các thư viện bằng câu lệnh sau
```
pip install dash
```

In [1]:
# Cài đặt các thư viện cần thiết
import dash
from dash import dcc
from dash import html
import pandas as pd
import plotly.graph_objs as go
from dash.dependencies import Input, Output
from keras.models import load_model as keras_load_model
import xgboost as xgb
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

import numpy as np

# Khởi tạo ứng dụng Dash
app = dash.Dash()
server = app.server

In [2]:
# Lấy dữ liệu EURUSD từ MetaTrader5
import MetaTrader5 as mt
import pandas as pd
from datetime import datetime

mt.initialize()

login = 84193298
password = "-8WyMrVz"
server = "MetaQuotes-Demo"

mt.login(login, password, server)

def get_data_FX(symbol, timeframe, rows):
    ohlc_data = pd.DataFrame(mt.copy_rates_from_pos(symbol, timeframe, 0, rows))
    ohlc_data['time'] = ohlc_data.apply(lambda x: datetime.fromtimestamp(x["time"]), axis=1)
    ohlc_data = ohlc_data.rename(columns={'time': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'close': 'Close', 'real_volume': 'Volume'})
    ohlc_data.set_index("Date")
    return ohlc_data



In [3]:
def get_predict_data(symbol, timeframe, rows, model_names, col_names):
    dataframe = get_data_FX(symbol, timeframe, rows)

    # Sắp xếp theo ngày, lấy cột ngày làm index
    dataframe["Date"]=pd.to_datetime(dataframe.Date,format="%Y-%m-%d")
    dataframe.index=dataframe['Date'].values

    dataframe=dataframe.sort_index(ascending=True,axis=0)

    # Tạo dataframe mới chứa dữ liệu cần thiết
    temp_df=pd.DataFrame(index=range(0,len(dataframe)),columns=['Date','Close', 'ROC'])

    for i in range(0,len(dataframe)):
        temp_df.loc[i, "Date"]=dataframe['Date'].iloc[i]
        temp_df.loc[i, "Close"]=dataframe["Close"].iloc[i]

    # Tính chỉ số ROC
    temp_df.loc[0, "ROC"] = None
    for i in range(1, len(dataframe)):
        temp_df.loc[i, "ROC"] = (temp_df.loc[i, "Close"] - temp_df.loc[i-1, "Close"]) / temp_df.loc[i-1, "Close"]

    temp_df.index=temp_df['Date'].values
    temp_df.drop("Date",axis=1,inplace=True)

    # Thêm 1 dòng vào cuối dữ liệu
    last_date = temp_df.index[-1]
    last_date = last_date + pd.DateOffset(minutes=1)
    temp_df.loc[last_date] = [None for _ in range(len(temp_df.columns))]

    dataframe = temp_df

    # Lấy mẫu từ dữ liệu để đưa ra dự đoán giá cổ phiếu bằng mô hình LSTM, RNN, XGB

    train=dataframe[:-20]
    valid=dataframe[-20:]

    for model_name in model_names:
        for col_name in col_names:
            if model_name == 'LSTM' or model_name == 'RNN':
                model = keras_load_model(f"model/{model_name.lower()}_{col_name.lower()}_model.keras")
            elif model_name == 'XGB':
                model = xgb.Booster()
                model.load_model(f"model/xgboost_{col_name.lower()}_model.json")

            inputs=temp_df[len(temp_df)-len(valid)-60:][col_name].values
            inputs=inputs.reshape(-1,1)

            scaler=MinMaxScaler(feature_range=(0,1))
            inputs=scaler.fit_transform(inputs)

            X_test=[]
            for i in range(60,inputs.shape[0]):
                X_test.append(inputs[i-60:i,0])
            X_test=np.array(X_test)


            if model_name == 'LSTM' or model_name == 'RNN':
                X_test_3D=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
                output=model.predict(X_test_3D)
                output=scaler.inverse_transform(output)
            elif model_name == 'XGB':
                X_test_DMatrix=xgb.DMatrix(X_test)
                output=model.predict(X_test_DMatrix).reshape(-1, 1)
                output=scaler.inverse_transform(output)

            # Thêm cột dự đoán vào dataframe
            dataframe.insert(dataframe.shape[1], f'Predictions_{col_name}_{model_name}', np.nan)
            for i in range(0, len(output)):
                dataframe.loc[dataframe.index[-len(output)+i], f'Predictions_{col_name}_{model_name}'] = output[i][0]

    return dataframe

model_names = ['LSTM', 'RNN', 'XGB']
col_names = ['Close', 'ROC']

dataframe = get_predict_data('EURUSD', mt.TIMEFRAME_M1, 2000, model_names, col_names)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 266ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 230ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 284ms/step


In [4]:
app.layout = html.Div([

    html.H1("Forex Price Analysis Dashboard", style={"textAlign": "center"}),

    dcc.Tabs(id="tabs", children=[

        dcc.Tab(label='EURUSD Actual closing price',children=[
			html.Div([
				html.H2("Actual closing price",style={"textAlign": "center"}),
				dcc.Graph(
					id="Actual Data",
					figure={
						"data":[
							go.Scatter(
								x=dataframe.index,
								y=dataframe["Close"],
								mode='markers'
							)

						],
						"layout":go.Layout(
							title='scatter plot',
							xaxis={'title':'Date'},
							yaxis={'title':'Closing Rate'}
						)
					}

				),
			])
        ]),

        dcc.Tab(label='EURUSD Prediction closing price', children=[
            html.Div([
                html.H1("Prediction closing price",
                        style={'textAlign': 'center'}),

                dcc.Dropdown(id='dropdown-close',
                             options=[
                                  {'label': 'EURUSD', 'value': 'EURUSD'},
                                  {'label': 'USDJPY', 'value': 'USDJPY'},
                                  {'label': 'GBPUSD', 'value': 'GBPUSD'}
                                  ],
                             multi=False,value='EURUSD',
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='prediction'),

            ]),
        ]),

        dcc.Tab(label='EURUSD Prediction ROC', children=[
            html.Div([
                html.H1("Prediction ROC",
                        style={'textAlign': 'center'}),

                dcc.Dropdown(id='dropdown-roc',
                             options=[
                                  {'label': 'EURUSD', 'value': 'EURUSD'},
                                  {'label': 'USDJPY', 'value': 'USDJPY'},
                                  {'label': 'GBPUSD', 'value': 'GBPUSD'}
                                  ],
                             multi=False,value='EURUSD',
                             style={"display": "block", "margin-left": "auto",
                                    "margin-right": "auto", "width": "60%"}),
                dcc.Graph(id='prediction_ROC'),

            ]),
        ])
        
    ])
])

@app.callback(Output('prediction', 'figure'),
              [Input('dropdown-close', 'value')])
def update_graph(selected_stock):
    selected_model = ["LSTM", "RNN", "XGB"]
    df = get_predict_data(selected_stock, mt.TIMEFRAME_M1, 2000, selected_model, ['Close','ROC'])
    model_names = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}
    trace1 = []
    trace2 = []
    
    trace1.append(
        go.Scatter(
            x=df.index, 
            y=df["Close"], 
            mode='lines', 
            opacity=0.6, 
            name="Actual Close Price", 
            marker_color='red',
            textposition='bottom center'
        )
    )

    for model in selected_model:
        trace2.append(
            go.Scatter(
                x=df.index,
				y=df["Predictions_Close_" + model],
                mode='lines', 
                opacity=0.7,
                name=f'Predictions_{model}',
                textposition='bottom center'
            )
        )

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]

    figure = {
        'data': data,
        'layout': go.Layout(colorway=
                            ["#FF0000", '#00FF00', '#00CC33', '#009966', '#006699', '#0033CC'],
        height=600,
        title=f"Close price for {', '.join(str(model_names[i]) for i in selected_model)} Over Time",
        xaxis={"title":"Date",
            'rangeselector': {
                'buttons': list([
                    {'count': 1, 'label': '1H','step': 'hour','stepmode': 'backward'},
                    {'count': 1, 'label': '1D','step': 'day','stepmode': 'backward'},
                    {'count': 7, 'label': '1W','step': 'day','stepmode': 'backward'},
                    {'count': 1, 'label': '1M','step': 'month','stepmode': 'backward'},
                    {'count': 6, 'label': '6M','step': 'month', 'stepmode': 'backward'},
                    {'step': 'all'}
                ])},
                'rangeslider': {'visible': True}, 'type': 'date'
            },
            yaxis={"title":f"Close {selected_stock}"})
    }
    return figure


@app.callback(Output('prediction_ROC', 'figure'),
              [Input('dropdown-roc', 'value')])
def update_graph(selected_stock):
    selected_model = ["LSTM", "RNN", "XGB"]
    df = get_predict_data(selected_stock, mt.TIMEFRAME_M1, 2000, selected_model, col_names)
    model_names = {"LSTM": "LSTM","RNN": "RNN", "XGB": "XGBoost"}
    trace1 = []
    trace2 = []
    
    trace1.append(
        go.Scatter(
            x=df.index, 
            y=df["ROC"], 
            mode='lines', 
            opacity=0.6, 
            name="Actual ROC", 
            marker_color='red',
            textposition='bottom center'
        )
    )

    for model in selected_model:
        trace2.append(
            go.Scatter(
                x=df.index,
				y=df["Predictions_ROC_" + model],
                mode='lines', 
                opacity=0.7,
                name=f'Predictions_{model}',
                textposition='bottom center'
            )
        )

    traces = [trace1, trace2]
    data = [val for sublist in traces for val in sublist]

    figure = {
        'data': data,
        'layout': go.Layout(colorway=
                            ["#FF0000", '#00FF00', '#00CC33', '#009966', '#006699', '#0033CC'],
        height=600,
        title=f"ROC for {', '.join(str(model_names[i]) for i in selected_model)} Over Time",
        xaxis={"title":"Date",
            'rangeselector': {
                'buttons': list([
                    {'count': 1, 'label': '1H','step': 'hour','stepmode': 'backward'},
                    {'count': 1, 'label': '1D','step': 'day','stepmode': 'backward'},
                    {'count': 7, 'label': '1W','step': 'day','stepmode': 'backward'},
                    {'count': 1, 'label': '1M','step': 'month','stepmode': 'backward'},
                    {'count': 6, 'label': '6M','step': 'month', 'stepmode': 'backward'},
                    {'step': 'all'}
                ])},
                'rangeslider': {'visible': True}, 'type': 'date'
            },
            yaxis={"title":f"ROC {selected_stock}"})
    }
    return figure

In [ ]:
if __name__=='__main__':
	app.run_server(debug=True)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 483ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 518ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 431ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 608ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 358ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 318ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 399ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 398ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 295ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 432ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 383ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step
